In [1]:
import pandas as pd
import os
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

In [2]:
# Parámetros del experimento
experimento = 3210

# Establecer la semilla aleatoria
semilla = 770767
random.seed(semilla)

In [3]:
# Parámetros de DecisionTreeClassifier
rpart_param = {
    "criterion": "gini",
    "min_samples_split": 300,
    "min_samples_leaf": 20,
    "max_depth": 10
}

In [4]:
# Parámetros del árbol
feature_fraction = 0.5
num_trees_max = 500

In [5]:
# Establecer el directorio de trabajo
os.chdir("/home/matias/Dropbox/ITBA/Mineria_de_Datos/dm2023b/")

In [6]:
# Cargar los datos
dataset = pd.read_csv("/home/matias/datasets/dataset_pequeno.csv")

/tmp/ipykernel_19773/3333360574.py:2: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("/home/matias/datasets/dataset_pequeno.csv")


In [7]:
# Crear la carpeta del experimento
carpeta_experimento = f"./exp/KA{experimento}/"
os.makedirs(carpeta_experimento, exist_ok=True)
os.chdir(carpeta_experimento)

In [8]:
# Definir los tamaños de ensemble a guardar en disco
grabar = [1, 5, 10, 50, 100, 200, 500]

In [9]:
# Definir conjuntos de entrenamiento y prueba
dtrain = dataset[dataset["foto_mes"] == 202107].copy()
dapply = dataset[dataset["foto_mes"] == 202109].copy()

In [10]:
# Inicializar la columna de probabilidad acumulada
dapply["prob_acumulada"] = 0

In [11]:
# Definir los campos utilizables para la predicción
campos_buenos = [col for col in dtrain.columns if col != "clase_ternaria"]


In [ ]:
# Generar las salidas
for arbolito in range(1, num_trees_max + 1):
    qty_campos_a_utilizar = int(len(campos_buenos) * feature_fraction)
    campos_random = random.sample(campos_buenos, qty_campos_a_utilizar)
    campos_random_str = " + ".join(campos_random)

    # Crear el árbol de decisión
    modelo = DecisionTreeClassifier(**rpart_param)
    modelo.fit(dtrain[campos_random], dtrain["clase_ternaria"])

    # Aplicar el modelo a los datos sin etiqueta
    prediccion = modelo.predict_proba(dapply[campos_random])[:, 1]

    dapply["prob_acumulada"] += prediccion

    if arbolito in grabar:
        # Generar la entrega para Kaggle
        umbral_corte = (1 / 40) * arbolito
        entrega = pd.DataFrame({
            "numero_de_cliente": dapply["numero_de_cliente"],
            "Predicted": (dapply["prob_acumulada"] > umbral_corte).astype(int)
        })

        nom_arch = f"KA{experimento}_{arbolito:03d}.csv"
        entrega.to_csv(nom_arch, index=False)
        print(arbolito, end=" ")